In [27]:
#import libraries and modules
import pandas as pd
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#from sklearn.metrics import classification_report
#from sklearn.pipeline import make_pipeline



#from sklearn.base import TransformerMixin
#class DenseTransformer(TransformerMixin):
#    
#    def fit(self, X, y=None, **fit_params):
#        return self
#    
#    def transform(self, X, y=None, **fit_params):
#        return X.todense()

from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import GaussianNB
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC


from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced


In [2]:
!pwd

/Users/Eric/Documents/DataAnalyticsBootcamp/MY_WORK_AREA/UTAUS201901DATA3/Projects/FinalProject/repo_world_wine_web_MY_SANDBOX/FinalProject-WorldWineWebML/ml_models


In [3]:
#import the data
data = pd.read_csv("final_wine_data_172k_test.csv")
data.head()

,Unnamed: 0,country,description,price,points,variety,winery
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",15.0,87,Portuguese Red,Quinta dos Avidagos
1,2,US,"Tart and snappy, the flavors of lime flesh and...",14.0,87,Pinot Gris,Rainstorm
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",13.0,87,Riesling,St. Julian
3,4,US,"Much like the regular bottling from 2012, this...",65.0,87,Pinot Noir,Sweet Cheeks
4,5,Spain,Blackberry and raspberry aromas show a typical...,15.0,87,Tempranillo-Merlot,Tandem


In [4]:
#filter just the country and descirption fields
data = data.filter(["country", "description"], axis = 1)
data.head()

,country,description
0,Portugal,"This is ripe and fruity, a wine that is smooth..."
1,US,"Tart and snappy, the flavors of lime flesh and..."
2,US,"Pineapple rind, lemon pith and orange blossom ..."
3,US,"Much like the regular bottling from 2012, this..."
4,Spain,Blackberry and raspberry aromas show a typical...


In [5]:
#look at the unique countries
data['country'].unique()

array(['Portugal', 'US', 'Spain', 'Italy', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', 'Turkey', 'Czech Republic', 'Slovenia', 'Luxembourg',
       'Croatia', 'Georgia', 'Uruguay', 'England', 'Lebanon', 'Serbia',
       'Brazil', 'Moldova', 'Morocco', 'Peru', 'India', 'Bulgaria',
       'Cyprus', 'Armenia', 'Switzerland', 'Bosnia and Herzegovina',
       'Ukraine', 'Slovakia', 'Macedonia', 'China', 'Kosovo', 'Global'],
      dtype=object)

In [6]:
#look at the country balance
data['country'].value_counts()

US                        70956
Italy                     31757
France                    26183
Spain                      8609
Portugal                   6913
Chile                      5284
Argentina                  4567
Austria                    3866
Australia                  3148
Germany                    2917
New Zealand                1935
South Africa               1582
Israel                      689
Greece                      566
Canada                      336
Uruguay                     228
Bulgaria                    202
Hungary                     187
Romania                     143
England                     123
Georgia                     113
Turkey                      107
Moldova                     100
Croatia                      85
Slovenia                     84
Brazil                       73
Mexico                       73
Lebanon                      35
Morocco                      34
Switzerland                  20
Peru                         17
Ukraine 

In [7]:
#peek at the description length
data['length'] = data['description'].apply(lambda x: len(x))
data.head()

,country,description,length
0,Portugal,"This is ripe and fruity, a wine that is smooth...",227
1,US,"Tart and snappy, the flavors of lime flesh and...",186
2,US,"Pineapple rind, lemon pith and orange blossom ...",199
3,US,"Much like the regular bottling from 2012, this...",249
4,Spain,Blackberry and raspberry aromas show a typical...,261


In [9]:
#look at the data types
data.dtypes

country        object
description    object
length          int64
dtype: object

In [16]:
# use Counter to get the top 20 wine countries

counter = Counter(data['country'].tolist())
top_5_countries = {i[0]: idx for idx, i in enumerate(counter.most_common(5))}
top5_countries_data = data[data['country'].map(lambda x: x in top_5_countries)]

top5_countries_data['country'].value_counts()

US          70956
Italy       31757
France      26183
Spain        8609
Portugal     6913
Name: country, dtype: int64

In [17]:
# filter out all extraneaous columns
df = top5_countries_data.filter(["country", "description"], axis = 1)
df.head()

,country,description
0,Portugal,"This is ripe and fruity, a wine that is smooth..."
1,US,"Tart and snappy, the flavors of lime flesh and..."
2,US,"Pineapple rind, lemon pith and orange blossom ..."
3,US,"Much like the regular bottling from 2012, this..."
4,Spain,Blackberry and raspberry aromas show a typical...


In [18]:
data = df

In [19]:
data.head()

,country,description
0,Portugal,"This is ripe and fruity, a wine that is smooth..."
1,US,"Tart and snappy, the flavors of lime flesh and..."
2,US,"Pineapple rind, lemon pith and orange blossom ..."
3,US,"Much like the regular bottling from 2012, this..."
4,Spain,Blackberry and raspberry aromas show a typical...


In [20]:
X=data['description']
y=data['country']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## Modeling

### Categorical Naive Bayes

In [ ]:
#undersampling

In [23]:
%%time

naive_bayes = make_pipeline_imb(
    CountVectorizer(stop_words='english', binary=True),
    NearMiss(),
    MultinomialNB()
)

naive_bayes.fit(X_train, y_train)

print(f'Accuracy: {naive_bayes.score(X_test, y_test)} \n')
print(classification_report_imbalanced(y_test, naive_bayes.predict(X_test)))

Accuracy: 0.7716728984905138 

                   pre       rec       spe        f1       geo       iba       sup

     France       0.87      0.47      0.98      0.61      0.68      0.44      5238
      Italy       0.73      0.98      0.90      0.84      0.94      0.89      6204
   Portugal       0.34      0.78      0.92      0.48      0.85      0.71      1414
      Spain       0.57      0.99      0.95      0.72      0.97      0.94      1777
         US       0.96      0.76      0.97      0.85      0.86      0.72     14251

avg / total       0.84      0.77      0.95      0.78      0.85      0.72     28884

CPU times: user 44.2 s, sys: 9.04 s, total: 53.3 s
Wall time: 43.2 s


In [ ]:
#oversampling

In [24]:
%%time

naive_bayes = make_pipeline_imb(
    CountVectorizer(stop_words='english', binary=True),
    SMOTE(),
    MultinomialNB()
)

naive_bayes.fit(X_train, y_train)

print(f'Accuracy: {naive_bayes.score(X_test, y_test)} \n')
print(classification_report_imbalanced(y_test, naive_bayes.predict(X_test)))

Accuracy: 0.8952361168813183 

                   pre       rec       spe        f1       geo       iba       sup

     France       0.87      0.79      0.97      0.83      0.88      0.76      5238
      Italy       1.00      0.83      1.00      0.91      0.91      0.82      6204
   Portugal       0.67      0.67      0.98      0.67      0.81      0.64      1414
      Spain       0.89      0.92      0.99      0.91      0.96      0.91      1777
         US       0.89      0.98      0.88      0.93      0.93      0.87     14251

avg / total       0.90      0.90      0.93      0.89      0.91      0.83     28884

CPU times: user 4min 39s, sys: 19.7 s, total: 4min 59s
Wall time: 4min 42s


### didnt wait--Gaussian Naive Bayes

### Categorical Bigram Naive Bayes

In [ ]:
%%time

bigram_naive_bayes = make_pipeline_imb(
    CountVectorizer(
        stop_words='english',
        binary=True,
        ngram_range=(1, 2)
    ),
    NearMiss(),
    MultinomialNB()
)

bigram_naive_bayes.fit(X_train, y_train)

print(f'Accuracy: {bigram_naive_bayes.score(X_test, y_test)} \n')
print(classification_report_imbalanced(y_test, bigram_naive_bayes.predict(X_test)))

### Word Count Logistic Regression

In [26]:
%%time

normalized_lr = make_pipeline_imb(
    CountVectorizer(stop_words='english'),
    Normalizer(),
    NearMiss(),
    LogisticRegression()
)

normalized_lr.fit(X_train, y_train)

print(f'Accuracy: {normalized_lr.score(X_test, y_test)} \n')
print(classification_report_imbalanced(y_test, normalized_lr.predict(X_test)))

NameError: name 'Normalizer' is not defined

### TF-IDF Logistic Regression Model

In [29]:
%%time

# TF-IDF Logistic Regression Model

tfidf_logistic_regression = make_pipeline_imb(
   CountVectorizer(strip_accents='ascii',
                   stop_words='english'),
   TfidfTransformer(),
    NearMiss(),
   LogisticRegression(),
   
   
)

tfidf_logistic_regression.fit(X_train, y_train)

print(f'Accuracy: {tfidf_logistic_regression.score(X_test, y_test)} \n')
print(classification_report_imbalanced(y_test, tfidf_logistic_regression.predict(X_test)))

/Users/Eric/anaconda3/envs/ut_ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Eric/anaconda3/envs/ut_ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.8511632737847944 

                   pre       rec       spe        f1       geo       iba       sup

     France       0.90      0.52      0.99      0.66      0.72      0.49      5238
      Italy       0.90      0.96      0.97      0.93      0.97      0.93      6204
   Portugal       0.47      0.76      0.96      0.58      0.86      0.72      1414
      Spain       0.65      0.97      0.97      0.78      0.97      0.94      1777
         US       0.92      0.92      0.92      0.92      0.92      0.84     14251

avg / total       0.87      0.85      0.95      0.85      0.89      0.80     28884

CPU times: user 47.4 s, sys: 8.54 s, total: 56 s
Wall time: 42.6 s


### Normalized Word Count Logistic Regression

In [ ]:
%%time

normalized_lr = make_pipeline(
    CountVectorizer(stop_words='english'),
    Normalizer(),
    LogisticRegression()
)

normalized_lr.fit(X_train, y_train)

print(f'Accuracy: {normalized_lr.score(X_test, y_test)} \n')
print(classification_report(y_test, normalized_lr.predict(X_test)))

### Bigram TF-IDF Logistic Regression

In [ ]:
%%time

bigram_tfidf_logistic_regression = make_pipeline(
    CountVectorizer(
        stop_words='english',
        ngram_range=(1,2)
    ),
    TfidfTransformer(),
    LogisticRegression()
)

bigram_tfidf_logistic_regression.fit(X_train, y_train)

print(f'Accuracy: {bigram_tfidf_logistic_regression.score(X_test, y_test)} \n')
print(classification_report(y_test, bigram_tfidf_logistic_regression.predict(X_test)))

###  TF-IDF Logistic Regression with Automatic Corpus Specific Stop Words

In [ ]:
%%time

mindf_max_tfidf_lr = make_pipeline(
    CountVectorizer(
        stop_words='english',
        min_df=2,
        max_df=.9
    ),
    TfidfTransformer(),
    LogisticRegression()
)

mindf_max_tfidf_lr.fit(X_train, y_train)

print(f'Accuracy: {mindf_max_tfidf_lr.score(X_test, y_test)} \n')
print(classification_report(y_test, mindf_max_tfidf_lr.predict(X_test)))



### caused reboot-Linear Discrete Analysis

### TF-IDF Linear SVC

In [ ]:
%%time

tfidf_svc = make_pipeline(
    CountVectorizer(
        stop_words='english',
        #min_df = 2,      # ed
        #max_df = .2      # ed
    ),
    TfidfTransformer(),
    #DenseTransformer(),
    LinearSVC()
)

tfidf_svc.fit(X_train, y_train)

print(f'Accuracy: {tfidf_svc.score(X_test, y_test)} \n')
print(classification_report(y_test, tfidf_svc.predict(X_test)))

### warnings--dnf   TF-IDF Logistic Lasso

### coef do not converge -- TF-IDF Logistic Ridge

### error-TF-IDF Logistic Elasticnet

### Looking at Top 20 Countries with top 3 accurate Models from above full data set work

In [ ]:
# use Counter to get the top 20 wine countries

counter = Counter(data['country'].tolist())
top_20_countries = {i[0]: idx for idx, i in enumerate(counter.most_common(20))}
top20_countries_data = data[data['country'].map(lambda x: x in top_20_countries)]

top20_countries_data['country'].value_counts()

In [ ]:
# filter out all extraneaous columns

df = top20_countries_data.filter(["country", "description"], axis = 1)
df.head()

In [ ]:
df.count()

In [ ]:
# split the dataset into training and testing datasets

X=df['description']
y=df['country']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

### TF-IDF Logistic Regression Model:  Pvs Accuracy: 0.8575353677072373  <<<<<<<<  #3


In [ ]:
%%time

# TF-IDF Logistic Regression Model

tfidf_logistic_regression = make_pipeline(
   CountVectorizer(strip_accents='ascii',
                   stop_words='english'),
   TfidfTransformer(),
   LogisticRegression(),
      
)

tfidf_logistic_regression.fit(X_train, y_train)

print(f'Accuracy: {tfidf_logistic_regression.score(X_test, y_test)} \n')
print(classification_report(y_test, tfidf_logistic_regression.predict(X_test)))

###  TF-IDF Logistic Regression with Automatic Corpus Specific Stop Words:  Pvs Accuracy: 0.8578861218285981   <<<<<<<  #2   

In [ ]:
%%time

mindf_max_tfidf_lr = make_pipeline(
    CountVectorizer(
        stop_words='english',
        min_df=2,
        max_df=.9
    ),
    TfidfTransformer(),
    LogisticRegression()
)

mindf_max_tfidf_lr.fit(X_train, y_train)

print(f'Accuracy: {mindf_max_tfidf_lr.score(X_test, y_test)} \n')
print(classification_report(y_test, mindf_max_tfidf_lr.predict(X_test)))


### TF-IDF Linear SVC:  Pvs  Accuracy: 0.881942008651935     <<<<<   #1


In [ ]:
%%time

tfidf_svc = make_pipeline(
    CountVectorizer(
        stop_words='english',
        #min_df = 2,      # ed
        #max_df = .2      # ed
    ),
    TfidfTransformer(),
    #DenseTransformer(),
    LinearSVC()
)

tfidf_svc.fit(X_train, y_train)

print(f'Accuracy: {tfidf_svc.score(X_test, y_test)} \n')
print(classification_report(y_test, tfidf_svc.predict(X_test)))

# Balanced Models

In [ ]:

## Use SMOTE oversampling
smote_pipeline = make_pipeline_imb(SMOTE(random_state=42), classifier(random_state=42))
smote_model = smote_pipeline.fit(X_train, y_train)
smote_prediction = smote_model.predict(X_test)